In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__valeurs_de_reference WHERE categorie_1='nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,categorie_1,categorie_2,categorie_3,limite_qualite,limite_qualite_unite,valeur_sanitaire_1,valeur_sanitaire_1_unite,valeur_sanitaire_2,valeur_sanitaire_2_unite,nb_rows
NO3,nitrate,None,None,50.0,mg/L,None,None,None,None,1
NO2,nitrate,None,None,0.5,mg/L,None,None,None,None,1
NO3_NO2,nitrate,None,None,1.0,None,None,None,None,None,1


In [3]:
%%sql
SELECT
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3_NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3_no2,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no2
FROM
    int__valeurs_de_reference
WHERE
    categorie_1 = 'nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

limite_qualite_no3,limite_qualite_no3_no2,limite_qualite_no2
50.0,1.0,0.5


**Les règles choisies**

Pour le choix du dernier prélèvement   (date et id) : on se réfère uniquement à nitrate (no3) 
Mais on doit regarder ensuite toutes les données  (no3/no2) des prélèvements sélectionnés

- Si on n'a pas nitrate (no3) :  pas de données dans la table. ca sera "pas recherché" sur le site
- Si nitrates (no3) ET nitrites (no2) 
    - Si Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L  --> inf_limite_qualite
    - Si Nitrates >= 50 mg/L et/ou nitrites >= 0,5 mg/L et/ou nitrate/50 + nitrites/3 >= 1 mg/L --> sup_limite_qualite
- Si nitrates (no3)  et pas nitrites (no2) :  
    -  Nitrates < 50 mg/L  --> inf_limite_qualite 
    -  Si Nitrates >= 50 mg/L -> sup_limite_qualite

In [4]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        referenceprel,
        datetimeprel,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND cdparametresiseeaux = 'NO3'
        AND -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
        AND valtraduite IS NOT NULL 
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
),

split_nitrites AS (
    SELECT
        resultats.cdreseau,
        resultats.categorie,
        resultats.datetimeprel AS dernier_prel_datetime,
        COUNT(DISTINCT resultats.cdparametresiseeaux) AS nb_parametres,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO3'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO2'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO3_NO2'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    LEFT JOIN
        int__resultats_udi_communes AS resultats
        ON
            resultats.cdreseau = last_pvl.cdreseau
            AND resultats.referenceprel = last_pvl.referenceprel
    WHERE
        last_pvl.row_number = 1 
        -- on ne veut que le dernier prélèvement
        AND resultats.categorie = 'nitrate' 
        -- on ne veut que les résultats associé aux nitrates/nitrites
    GROUP BY
        resultats.cdreseau,
        resultats.categorie,
        resultats.datetimeprel
),

split_nitrites_with_ref AS (
    SELECT
        split_nitrites.cdreseau,
        split_nitrites.categorie,
        split_nitrites.nb_parametres,
        split_nitrites.dernier_prel_datetime,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no2,
        valeur_ref.limite_qualite_no3_no2,
        split_nitrites.valtraduite_no3_no2,
        split_nitrites.valtraduite_no3,
        split_nitrites.valtraduite_no2
    FROM
        split_nitrites
    CROSS JOIN
        valeur_ref
)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
    CASE
        WHEN -- Si nitrates (no3) et pas nitrites (no2)
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL 
            AND split_nitrites_with_ref.valtraduite_no2 IS  NULL
            AND
            split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            THEN 'inf_limite_qualite'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL 
            AND split_nitrites_with_ref.valtraduite_no2 IS  NULL
            AND
            split_nitrites_with_ref.valtraduite_no3
            >= split_nitrites_with_ref.limite_qualite_no3
            THEN 'sup_limite_qualite'
        WHEN -- Si nitrates (no3) ET nitrites (no2)
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no2 IS NOT NULL
            AND (
                split_nitrites_with_ref.valtraduite_no3
                >= split_nitrites_with_ref.limite_qualite_no3
                OR split_nitrites_with_ref.valtraduite_no2
                >= split_nitrites_with_ref.limite_qualite_no2
                OR COALESCE(
                    split_nitrites_with_ref.valtraduite_no3_no2,
                    (
                        split_nitrites_with_ref.valtraduite_no3 / 50
                        + split_nitrites_with_ref.valtraduite_no2 / 3
                    )
                )
                >= split_nitrites_with_ref.limite_qualite_no3_no2
            )
            THEN 'sup_limite_qualite'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no2 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            AND split_nitrites_with_ref.valtraduite_no2
            < split_nitrites_with_ref.limite_qualite_no2
            AND COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                (
                    split_nitrites_with_ref.valtraduite_no3 / 50
                    + split_nitrites_with_ref.valtraduite_no2 / 3
                )
            )
            < split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'inf_limite_qualite'
        ELSE 'erreur'
    END AS resultat
FROM
    split_nitrites_with_ref


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
043000382,nitrate,dernier_prel,2024-08-21 11:04:00,3,inf_limite_qualite
048000421,nitrate,dernier_prel,2024-05-23 12:04:00,2,inf_limite_qualite
055000098,nitrate,dernier_prel,2024-08-05 08:48:00,3,inf_limite_qualite
066000236,nitrate,dernier_prel,2024-06-19 10:33:00,2,inf_limite_qualite
066000312,nitrate,dernier_prel,2024-10-29 10:32:00,2,inf_limite_qualite
067001856,nitrate,dernier_prel,2024-12-06 08:15:00,3,inf_limite_qualite
068001002,nitrate,dernier_prel,2024-10-28 09:40:00,2,inf_limite_qualite
073001318,nitrate,dernier_prel,2024-12-19 09:35:00,3,inf_limite_qualite
073001176,nitrate,dernier_prel,2024-08-28 11:59:00,3,inf_limite_qualite
080000913,nitrate,dernier_prel,2024-11-28 14:38:00,3,inf_limite_qualite


In [5]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, COUNT(*)
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,count_star()
inf_limite_qualite,21184
sup_limite_qualite,224


In [6]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier
WHERE 
    resultat='erreur'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


####  Cf cas cdreseau IN( '034005906') , referenceprel= 03400327764

In [8]:
%%sql
    SELECT
*,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND cdparametresiseeaux = 'NO3'
        AND -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
        AND cdreseau IN( '034005906')

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,de_partition,valtraduite,categorie,categorie_2,categorie_3,limite_qualite,valeur_sanitaire_1,valeur_sanitaire_2,cdreseau,inseecommune,datetimeprel,row_number
03400327273,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34088,2025-01-31 09:34:00,9
03400327273,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34087,2025-01-31 09:34:00,10
03400327275,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34088,2025-01-28 07:50:00,11
03400327275,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34087,2025-01-28 07:50:00,12
03400327272,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34088,2025-01-08 10:17:00,13
03400327272,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34087,2025-01-08 10:17:00,14
03400316443,NO3,2024,3.000,nitrate,None,None,50.0,None,None,034005906,34087,2024-12-03 09:23:00,15
03400316443,NO3,2024,3.000,nitrate,None,None,50.0,None,None,034005906,34088,2024-12-03 09:23:00,16
03400327764,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34088,2025-02-21 10:16:00,1
03400327764,NO3,2025,None,nitrate,None,None,50.0,None,None,034005906,34087,2025-02-21 10:16:00,2


In [9]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min_cdreseau,max_cdreseau
sup_limite_qualite,002000060,089003505
inf_limite_qualite,001000003,976003881


# Check 

In [11]:
%%sql
WITH
last_pvl AS (
    SELECT
        cdreseau,
        referenceprel,
        datetimeprel,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND cdparametresiseeaux = 'NO3'
        AND -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
    AND valtraduite IS NOT NULL 
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
),

split_nitrites AS (
    SELECT
        resultats.cdreseau,
        resultats.categorie,
        resultats.datetimeprel AS dernier_prel_datetime,
        COUNT(DISTINCT resultats.cdparametresiseeaux) AS nb_parametres,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO3'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO2'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    resultats.cdparametresiseeaux = 'NO3_NO2'
                    THEN resultats.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    LEFT JOIN
        int__resultats_udi_communes AS resultats
        ON
            resultats.cdreseau = last_pvl.cdreseau
            AND resultats.referenceprel = last_pvl.referenceprel
    WHERE
        last_pvl.row_number = 1 
        -- on ne veut que le dernier prélèvement
        AND resultats.categorie = 'nitrate' 
        -- on ne veut que les résultats associé aux nitrates/nitrites
    GROUP BY
        resultats.cdreseau,
        resultats.categorie,
        resultats.datetimeprel
),

split_nitrites_with_ref AS (
    SELECT
        split_nitrites.cdreseau,
        split_nitrites.categorie,
        split_nitrites.nb_parametres,
        split_nitrites.dernier_prel_datetime,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no2,
        valeur_ref.limite_qualite_no3_no2,
        split_nitrites.valtraduite_no3_no2,
        split_nitrites.valtraduite_no3,
        split_nitrites.valtraduite_no2
    FROM
        split_nitrites
    CROSS JOIN
        valeur_ref
) 
    
SELECT 
    *,
    CASE
        WHEN -- Si nitrates (no3) et pas nitrites (no2)
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL 
            AND split_nitrites_with_ref.valtraduite_no2 IS  NULL
            AND
            split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            THEN 'inf_limite_qualite'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL 
            AND split_nitrites_with_ref.valtraduite_no2 IS  NULL
            AND
            split_nitrites_with_ref.valtraduite_no3
            >= split_nitrites_with_ref.limite_qualite_no3
            THEN 'sup_limite_qualite'
        WHEN -- Si nitrates (no3) ET nitrites (no2)
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no2 IS NOT NULL
            AND (
                split_nitrites_with_ref.valtraduite_no3
                >= split_nitrites_with_ref.limite_qualite_no3
                OR split_nitrites_with_ref.valtraduite_no2
                >= split_nitrites_with_ref.limite_qualite_no2
                OR COALESCE(
                    split_nitrites_with_ref.valtraduite_no3_no2,
                    (
                        split_nitrites_with_ref.valtraduite_no3 / 50
                        + split_nitrites_with_ref.valtraduite_no2 / 3
                    )
                )
                >= split_nitrites_with_ref.limite_qualite_no3_no2
            )
            THEN 'sup_limite_qualite'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no2 IS NOT NULL
            AND split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            AND split_nitrites_with_ref.valtraduite_no2
            < split_nitrites_with_ref.limite_qualite_no2
            AND COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                (
                    split_nitrites_with_ref.valtraduite_no3 / 50
                    + split_nitrites_with_ref.valtraduite_no2 / 3
                )
            )
            < split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'inf_limite_qualite'
        ELSE 'erreur'
    END AS resultat
FROM
    split_nitrites_with_ref
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293','008000681','008000546')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,nb_parametres,dernier_prel_datetime,limite_qualite_no3,limite_qualite_no2,limite_qualite_no3_no2,valtraduite_no3_no2,valtraduite_no3,valtraduite_no2,resultat
001000003,nitrate,3,2025-01-21 12:35:00,50.0,0.5,1.0,0.280,14.000,0.000,inf_limite_qualite
001000260,nitrate,3,2024-12-06 12:02:00,50.0,0.5,1.0,0.050,2.400,0.000,inf_limite_qualite
001000268,nitrate,3,2025-01-13 09:57:00,50.0,0.5,1.0,0.120,6.000,0.000,inf_limite_qualite
001000293,nitrate,3,2025-01-28 11:29:00,50.0,0.5,1.0,0.070,3.700,0.000,inf_limite_qualite
002000060,nitrate,2,2024-10-18 08:50:00,50.0,0.5,1.0,None,59.300,0.000,sup_limite_qualite
095000346,nitrate,1,2025-02-27 08:35:00,50.0,0.5,1.0,None,47.000,None,inf_limite_qualite
973000028,nitrate,2,2024-12-02 11:30:00,50.0,0.5,1.0,None,0.000,0.000,inf_limite_qualite


In [14]:
%%sql --with int__resultats_nitrites_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_nitrites_udi_dernier
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293','008000681','008000546')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,inf_limite_qualite
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,inf_limite_qualite
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,inf_limite_qualite
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,inf_limite_qualite
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,2,sup_limite_qualite
008000546,nitrate,dernier_prel,2025-02-07 10:38:00,1,sup_limite_qualite
008000681,nitrate,dernier_prel,2025-02-12 11:38:00,1,sup_limite_qualite
095000346,nitrate,dernier_prel,2025-02-27 08:35:00,1,inf_limite_qualite
973000028,nitrate,dernier_prel,2024-12-02 11:30:00,2,inf_limite_qualite


In [15]:
%%sql --with int__resultats_nitrites_udi_dernier
-- dernier relevé
SELECT
*
FROM
    int__resultats_nitrites_udi_dernier
WHERE
    (
        cdreseau = '001000003'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-01-21 12:35:00'
        AND resultat != 'inf_limite_qualite'
    )
    OR
    (
        cdreseau = '001000260'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-06 12:02:00'
        AND resultat != 'inf_limite_qualite'
    )
    OR
    (
        cdreseau = '001000268'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-01-13 09:57:00'
        AND resultat != 'inf_limite_qualite'
    )
    OR
    (
        cdreseau = '002000060'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-10-18 08:50:00'
        AND resultat != 'sup_limite_qualite'
    )
    OR
    (
        cdreseau = '095000346'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-27 08:35:00'
        AND resultat != 'inf_limite_qualite'
    )
    OR
    (
        cdreseau = '973000028'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-02 11:30:00'
        AND resultat != 'inf_limite_qualite'
    )
    OR
    (
        cdreseau = '008000681'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-12 11:38:00'
        AND resultat != 'sup_limite_qualite'
    )
    OR
    (
        cdreseau = '008000546'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-07 10:38:00'
        AND resultat != 'sup_limite_qualite'
    )

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
